In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

### Exploretory Analysis
### 1. General Overview

In [ ]:
file = pd.read_sas("a2z_insurance.sas7bdat")
file.set_index("CustID", inplace=True)
file.head()

In [ ]:
file.describe(include="all").T

In [ ]:
file[~file.BirthYear.isnull()]["BirthYear"].sort_values()


In [ ]:
file[~file.FirstPolYear.isnull()]["FirstPolYear"].sort_values()


In [ ]:
file.dtypes

In [ ]:
file.info();

### 2. Seperation in Metric & NonMetric Features

In [ ]:
metric_cols = ["FirstPolYear","BirthYear","MonthSal","CustMonVal","ClaimsRate","PremMotor","PremHousehold",
               "PremHealth","PremLife","PremWork"]
nonmetric_cols = ["EducDeg","GeoLivArea","Children"]


#Assign Datatypes -> EducDeg Categorical logical order

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from math import ceil
# All Numeric Variables' Box Plots in one figure
sns.set()

# Prepare figure. Create individual axes where each box plot will be placed
fig, axes = plt.subplots(2, ceil(len(metric_cols) / 2), figsize=(20, 11))

# Plot data
# Iterate across axes objects and associate each box plot (hint: use the ax argument):
for ax, feat in zip(axes.flatten(), metric_cols): # Notice the zip() function and flatten() method
    sns.violinplot(x=file[feat], ax=ax)
    
# Layout
# Add a centered title to the figure:
title = "Numeric Features' Violin Plots"

plt.suptitle(title)

plt.show()

### 3. Unique value count for NonMetric Feature

In [ ]:
import matplotlib.pyplot as plt
NonMetricFile = file[nonmetric_cols]
Countdict = dict()
for col in NonMetricFile.columns:
    Countdict[str(col)] = len(NonMetricFile[str(col)].dropna().unique())
lel = pd.DataFrame.from_dict(Countdict, orient='index', columns= ["Count"])
lel
ax = lel.plot(kind = "bar",ylabel='count',
         xlabel='Degree', figsize=(6, 5), colormap='Paired')

In [ ]:
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
data = file[metric_cols]
scaler = MinMaxScaler()
data[metric_cols] = scaler.fit_transform(data)
sns.pairplot(data);

### 4. Data Quality Assessment

#### I. Missing Values

In [ ]:
file.isnull().sum().sum()

In [ ]:
file["Missing_Values"] = file.isnull().sum(axis=1).to_list()
sefs = file.groupby("Missing_Values")[["Missing_Values"]].count().rename(columns={"Missing_Values":'Missing_Values_count'}).reset_index()

In [ ]:
file["Missing_Values"] = file.isnull().sum(axis=1).to_list()
sefs = file.groupby("Missing_Values")[["Missing_Values"]].count().rename(columns={"Missing_Values":'Missing_Values_count'}).reset_index()
sefs1 = sefs[sefs.Missing_Values > 0]
plt.bar("Missing_Values","Missing_Values_count",data = sefs1)
plt.xlabel("Amount_Missing_Values")
xlocs = plt.xticks()
xlocs = [x for x in range(1,len(sefs1)+1)]
plt.xticks(xlocs)
for i, v in enumerate(sefs1.Missing_Values_count.to_list()):
    plt.text(xlocs[i]- 0.1, v+4, str(v))

#### II. Uniqueness

In [ ]:
file[file.duplicated(keep=False)==True]


In [ ]:
file.sort_values(["CustMonVal","ClaimsRate"])

### 5. Relation between Features

In [ ]:
file.corr()
plt.figure(figsize=(10,10))
corr = file.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True, annot=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)
ax.set_yticklabels(
    ax.get_yticklabels(),
    rotation=45,
);

In [ ]:
file

### Reconstructering FirstPolYear

##### Aquisitioncost = 25?!

In [ ]:
file

In [ ]:
test12 = file.copy()
test12[test12.Missing_Values >= test12.Missing_Values.max()]
Aquisitioncost = test12[test12.Missing_Values >= test12.Missing_Values.max()]["CustMonVal"].iloc[0]
print(f"Aquisitioncost: {Aquisitioncost}")

##### Reconstructering Part

In [ ]:
test12 = test12[test12.ClaimsRate == 0]
test12["SumPrem"] = test12.PremMotor+test12.PremHousehold+test12.PremHealth+test12.PremLife+test12.PremWork

test12["NewFirstPolYear"] = (test12.CustMonVal-25) / ((test12.SumPrem) - (test12.ClaimsRate * test12.SumPrem))

In [ ]:
metric_cols.remove('FirstPolYear')
file.drop(columns = "FirstPolYear",inplace=True)

In [ ]:
file.isnull().sum().sum()

### Data Preprocessing

In [ ]:
# Plausibilitycheck 3.2.1 in Report 
# here we know that these are two observations that having unplausible values, therefor we set them to nan and impute them later
#file['FirstPolYear'] = np.where(file['FirstPolYear'] > 2016, np.NaN, file['FirstPolYear'])
file['BirthYear'] = np.where(file['BirthYear'] < 1900, np.NaN, file['BirthYear'])
prepro_df = file.copy()

#### Outlier

Univariate Outlier Detection Methods

In [ ]:
def remove_outlier_IQR(df,factor=1.5):
    df_final=df.copy()
    Q1 = df_final.quantile(0.25)
    Q3 = df_final.quantile(0.75)
    IQR = Q3 - Q1 #Every data point between Q1 and Q3 are inside the interquartile range.
    return df_final[~((df_final < (Q1 - 1.5 * IQR)) | (df_final > (Q3 + 1.5 * IQR))).any(axis=1)]


removed_iqr = remove_outlier_IQR(file,factor=1.5)

In [ ]:
#Made by Adriana:

def remove_outliers_zscore(df,metric_ft):
    
    #fill nan with median of column
    data = df.copy()
    
    for col in metric_ft:
            data[col].fillna(data[col].median(),inplace=True)
    
    #Normalize the metric features of the data so that differetn scales dont influence the distance metrics that DBScan uses
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler() #Normalize data with min max scaler
    df_normalize = scaler.fit_transform(data[metric_ft])

   #fit zscore to the data
    for col in data[metric_ft]:
        col_zscore = col + '_zscore'
        data['zscore'] = (data[col]-data[col].mean())/data[col].std(ddof=0)
    
    # clean datafame, without outliers
    data_outliers = data[abs(data['zscore']>3)]
    df_final = pd.concat([data, data_outliers, data_outliers]).drop_duplicates(keep=False)
  
    return df_final

removedzScore = remove_outliers_zscore(file,metric_cols)

Multivariate Outlier Detecion Methods

In [ ]:
def remove_outliers_DBScan(df,metric_ft,nonmetric_columns,eps=0.027,mi_sample = 5):
    
    #fill nan with median of column
    data = df.copy()
    
    for col in metric_ft:
            data[col].fillna(data[col].median(),inplace=True)
    
    #Normalize the metric features of the data so that differetn scales dont influence the distance metrics that DBScan uses
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler() #Normalize data with min max scaler
    df_normalize = scaler.fit_transform(data[metric_ft])

   #fit DBScan to the data
    from sklearn.cluster import DBSCAN
    outlier_detection = DBSCAN(
      eps = eps,
      metric="euclidean",
      min_samples = mi_sample,
      n_jobs = -1)
    clusters = outlier_detection.fit_predict(df_normalize)
    
    unique, counts = np.unique(clusters, return_counts=True)
    print(dict(zip(unique, counts)))
    data["cluster"] = clusters
    return df[data["cluster"]!= -1], dict(zip(unique, counts))


In [ ]:
# #Tune epsilon hyperparameter for DBScan outlier detection

# outliernr=list()
# epsl=list()
# clusternr=list()

# #try epsilon values between 0.001 and 0.3
# for epsil in tqdm(np.arange(0.001, 0.3, 0.001)):
#     _,dicti = remove_outliers_DBScan(file,metric_cols,nonmetric_cols,epsil,20)
    
#     outliernr.append(dicti[-1])
#     epsl.append(epsil)
#     if len(dicti.keys()) > 2: #Check if there is more than 1 cluster by checking the number of keys in the dictionary.
#         clusternr.append(">2")
#     else:
#         clusternr.append("2")

In [ ]:
# sns.scatterplot(x=epsl[11:-120],y=outliernr[11:-120],hue=clusternr[11:-120],edgecolor="none")
# plt.title("#Detected Outliers for different epsilons")
# plt.xlabel("Epsilon")
# plt.ylabel("Number of detected Outliers")
# plt.xticks([epsl[31],epsl[-120],epsl[-120]/2])
# plt.legend(title='# Clusters')
# plt.show()

In [ ]:
#Code partly taken from https://towardsdatascience.com/multivariate-outlier-detection-in-python-e946cfc843b3

def mahalanobis_outlier(df,metric_ft,threshold= 0.97):
    data = df[metric_ft].copy()
    
    for col in metric_ft:
        data[col].fillna(data[col].median(), inplace = True)

    indexd = data.index
    
    data = data.to_numpy()

    covariance  = np.cov(data , rowvar=False)

    # Covariance matrix power of -1
    covariance_pm1 = np.linalg.matrix_power(covariance, -1)

    # Center point
    centerpoint = np.mean(data , axis=0)


    from scipy.stats import chi2
    # Distances between center point and 
    distances = []
    for i, val in enumerate(data):
          p1 = val
          p2 = centerpoint
          distance = (p1-p2).T.dot(covariance_pm1).dot(p1-p2)
          distances.append(distance)
    distances = np.array(distances)

    # Cutoff (threshold) value from Chi-Sqaure Distribution for detecting outliers 
    cutoff = chi2.ppf(threshold, data.shape[1])

    # Index of outliers
    outlierIndexes = np.where(distances > cutoff)
    
    pdD = pd.DataFrame(data,columns=metric_ft,index = indexd)
    pdD.loc[:,"distances"]=distances
    pdD.loc[:,"cutoff"]=cutoff
    
    pdD = pdD.query("distances < cutoff").drop(["distances","cutoff"],axis=1) #Select only rows in which the distance is smaller than the cutoff threshold
    df = df[df.index.isin(pdD.index.tolist())]
    return df

removed_mahalanobis = mahalanobis_outlier(file,metric_cols)


In [ ]:

def UMAD_Visualization(df,metric_features, cluster_col):
    import umap.umap_ as umap
    import plotly.express as px
    # This is step can be quite time consuming
    UMAP1 = umap.UMAP(n_components=3, init='random', random_state=38).fit_transform(df[metric_features])
    some_df = df[[str(cluster_col)]].astype(int)

    fig = px.scatter_3d(
        UMAP1, x=0, y=1, z=2,
        color=some_df[str(cluster_col)],width=900,
        height=900,labels={'color': 'cluster'}
    )
    fig.show()

In [ ]:
#We can vizualize 3 dimensions at max. Hence we reduce the dimensionality of the dataset down to 3 by using PCA, that we will descibe in more detail later on.
#Although we cannot capture the entire variance with this plot, but a significat amount is caputred by PC1,PC2,PC3

def vizualize_outlier_detect_UMAP(df_before, df_after):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    
    import umap.umap_ as umap
    import plotly.express as px
    #pca = PCA(n_components=3)
    UMAP1 = umap.UMAP(n_components=3, init='random', random_state=38)
    df= df_before.copy()
    for col in metric_cols:
        df[col].fillna(df[col].median(), inplace = True)
    
    
    #
    standard_scaled_df = pd.DataFrame(scaler.fit_transform(df[metric_cols]),columns=metric_cols,index=df[metric_cols].index)
    df_=pd.DataFrame(UMAP1.fit_transform(standard_scaled_df),columns=["D1","D2","D3"],index=df[metric_cols].index)

    df_.loc[df_after.index,"Outlier?"]="No"
    df_.loc[[el for el in df_before.index if el not in df_after.index],"Outlier?"]="Yes"

    import plotly.express as px
    fig = px.scatter_3d(df_, x='D1', y='D2', z='D3',
                  color='Outlier?',title="Visualization of Outliers using 3 Principle Components",width=900,
        height=900)
    
    fig.show()

In [ ]:
#We can vizualize 3 dimensions at max. Hence we reduce the dimensionality of the dataset down to 3 by using PCA, that we will descibe in more detail later on.
#Although we cannot capture the entire variance with this plot, but a significat amount is caputred by PC1,PC2,PC3

def vizualize_outlier_detect(df_before, df_after):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    
    from sklearn.decomposition import PCA
    pca = PCA(n_components=3)
    df= df_before.copy()
    for col in metric_cols:
        df[col].fillna(df[col].median(), inplace = True)
    
    #PCA requires the df to be scaled.
    standard_scaled_df = pd.DataFrame(scaler.fit_transform(df[metric_cols]),columns=metric_cols,index=df[metric_cols].index)
    df_pca=pd.DataFrame(pca.fit_transform(standard_scaled_df),columns=["PC1","PC2","PC3"],index=df[metric_cols].index)

    
    #Add a columns that says if a data point has been recognized as an outlier
    df_pca.loc[df_after.index,"Outlier?"]="No"
    df_pca.loc[[el for el in df_before.index if el not in df_after.index],"Outlier?"]="Yes"

    #Use plotly to make a 3d scatterplot 
    import plotly.express as px
    fig = px.scatter_3d(df_pca, x='PC1', y='PC2', z='PC3',
                  color='Outlier?',title="Visualization of Outliers using 3 Principle Components")
    
    fig.show()


In [ ]:
###Isolation Forest
def IsolationTreez3(df,metric_columns, contamination,*argv):
    ''' This function will identify outlier by using all columns that are at least ordinal scaled.
        Inputs: DataFrame, metric columns, contamination = Amount of expected Outliers and all additional ordinal columns'''
    import pandas as pd
    from sklearn.ensemble import IsolationForest
    import numpy as np
    from sklearn.preprocessing import LabelEncoder
    to_label = list()
    for arg in argv:
        to_label.append(arg)

    data = df[metric_columns].copy()
    for col in data.columns:
        data[col].fillna(data[col].median(),inplace=True)
    for col in to_label:
        data[col] = df[col]
        data[col].fillna(data[col].mode().iloc[0])
        label_encoder = LabelEncoder()
        integer_encoded = label_encoder.fit_transform(data[col])            
        data[col] = integer_encoded




    model = IsolationForest(n_estimators=200,max_samples=0.9,bootstrap = True, contamination=float(contamination),random_state=np.random.RandomState(49))
    model.fit(data.values)
    data["iforest"] = model.predict(data.values).tolist()
    print(data["iforest"].value_counts())
    
    
    df = df[~df.index.isin(data[data["iforest"]==-1].index.tolist())]
    return df.copy(),data[data["iforest"]==-1];

In [ ]:

removedIsolationForest,outlier = IsolationTreez3(prepro_df,metric_cols, 0.01)
removedDBscan,_ = remove_outliers_DBScan(prepro_df,metric_cols,nonmetric_cols,0.032,20) #0.032 was the result of hyperparametertuning
vizualize_outlier_detect(prepro_df,removedDBscan)
vizualize_outlier_detect(prepro_df,removedIsolationForest)
vizualize_outlier_detect(prepro_df,removed_iqr)

In [ ]:
vizualize_outlier_detect_UMAP(prepro_df,removedIsolationForest)
vizualize_outlier_detect_UMAP(prepro_df,removedDBscan)

In [ ]:
vizualize_outlier_detect(removedIsolationForest,removedIsolationForest)

In [ ]:
vizualize_outlier_detect(removedDBscan,removedDBscan)

### Missing Value Imputation

In [ ]:
misval_df = removedIsolationForest.copy()

In [ ]:
#  as we explained in 3.3 of our report, here we are replacing the nans the Premiumns of all rows with more than 4 missing Values
#  with 0
misval_df.loc[misval_df.Missing_Values == 4] = misval_df.loc[misval_df.Missing_Values == 4].fillna(0)

In [ ]:
#Nr of Rows with Missing Values after outlierremoval
len(misval_df[misval_df.Missing_Values >0].index.to_list())

In [ ]:
misval_df.drop("Missing_Values",1, inplace = True)

In [ ]:
#Simple Imputation of missing values of ht entire dataFrame with mode fpr nonmetric columns und median for metric columns

def simple_imputation(df,non_metric):
    data = df.copy()
    for col in df.columns:
        if col in non_metric:
            data[col].fillna(data[col].mode()[0], inplace = True)
        else:
            data[col].fillna(data[col].median(), inplace = True)
            
    return data

In [ ]:
def supervised_impute(df_without_outliers): 
    from sklearn import preprocessing
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeClassifier

    df_without_outliers["EducDeg"]= pd.Series(preprocessing.LabelEncoder().fit_transform(df_without_outliers.EducDeg)) #Label Encode Educ Degree

    discrete = ["EducDeg","Children","GeoLivArea"]
    continous = [a for a in df_without_outliers.columns if a not in discrete]

    from sklearn.preprocessing import MinMaxScaler #Scale data
    scaler = MinMaxScaler()
    scaled_df = pd.DataFrame(scaler.fit_transform(df_without_outliers[continous]),columns=continous,index=df_without_outliers[continous].index) #Bring all features to the same scale
    df_without_outliers = pd.concat([scaled_df, df_without_outliers[discrete]], axis=1)

    summary = dict()

    #Hard coded the best ccp_alphas for the decisiontree regressor to make this cell run faster. Please find the code that leads to these optimized alphas in the decision tree section
    best_ccp_alphas = {"BirthYear":2.2379104047158256e-05,"MonthSal":1.0322055686659169e-05,"PremMotor":1.5401288299580906e-06,"PremHealth":1.5401288299580906e-06,"PremLife":1.099183996339592e-05,"PremWork":6.7845242778510876e-06,"EducDeg":0.0004856187200293349,"Children":0.0005468937947268604,"GeoLivArea":0.0005179895310502686}
    
    for colu in tqdm(df_without_outliers.columns):

        if df_without_outliers[colu].isnull().sum() == 0: #We can skip a feature in case it has no missing values to impute
            continue

        simputed = simple_imputation(df_without_outliers,nonmetric_cols)
        simputed.loc[:,colu] = df_without_outliers[colu] #Copy back in the target columns with

        simputed_df_final = simputed[simputed[colu].notna()].copy() # drop all NaN rows, because those one we neither use for testing nor for training purposes

        X = simputed_df_final.drop(columns = colu)
        y = simputed_df_final.pop(colu)

        models = dict() #Store different models and their scores in dict

        if colu in continous: #Perform various Regression models on continous columns


            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 15)



            #Linear Regression
            from sklearn.linear_model import LinearRegression

            model = LinearRegression().fit(X_train, y_train)

            predictions = model.predict(X_test)

            from sklearn.metrics import explained_variance_score
            from sklearn.metrics import mean_squared_error

            models[model] = mean_squared_error(y_test.values, predictions)

            #-------------


            #KNN:
            from sklearn.neighbors import KNeighborsRegressor

            kn = dict()
            for k in range(2,40): # Tune k as a hyperparameter

                knn = KNeighborsRegressor(n_neighbors=k).fit(X_train, y_train)
                predictions = knn.predict(X_test)
                kn[knn] = mean_squared_error(y_test.values, predictions)

            models[min(kn, key=kn.get)] = min(kn.values())

            #-------------

            #Decision Tree...
            from sklearn.tree import DecisionTreeRegressor
            
            
            #deci = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)


            #path = deci.cost_complexity_pruning_path(X_train, y_train)
            #ccp_alphas, impurities = path.ccp_alphas, path.impurities

            #errorss=dict()
            #for alp in tqdm(ccp_alphas[::2]):
                #deci = DecisionTreeRegressor(random_state=0,ccp_alpha=alp).fit(X_train, y_train)
                #predictions = deci.predict(X_test)
                #errorss[alp]=mean_squared_error(y_test.values, predictions)



            #deci = DecisionTreeRegressor(random_state=0,ccp_alpha=min(errorss, key=errorss.get)).fit(X_train, y_train)
            deci = DecisionTreeRegressor(random_state=0,ccp_alpha=best_ccp_alphas[colu]).fit(X_train, y_train)

            predictions = deci.predict(X_test)
            models[deci] = mean_squared_error(y_test.values, predictions)


            #---------------
            

            simpl = np.full(y_test.shape, int(y_train.median()))
            models["simple"]=mean_squared_error(y_test.values, simpl)

            best_model = min(models, key=models.get)




        elif colu in discrete: #Perform various classification models on discrete features     

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18,stratify=y) #Use stratify to make sure the same proportion of dicrete values are in the train  and test set.

            from sklearn.metrics import accuracy_score

            model = LogisticRegression(max_iter=2000).fit(X_train, y_train)
            predictions = model.predict(X_test)
            models[model] = accuracy_score(y_test.values, predictions)

            #------

        # Here I pruned the decision trees using ccp_alpha from the cost complexity path
            #deci = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)


            #path = deci.cost_complexity_pruning_path(X_train, y_train)
            #ccp_alphas, impurities = path.ccp_alphas, path.impurities

            #scores=dict()
            #for alp in tqdm(ccp_alphas[::2]):
                #deci = DecisionTreeClassifier(random_state=0,ccp_alpha=alp).fit(X_train, y_train)
                #predictions = deci.predict(X_test)
                #scores[alp]=accuracy_score(y_test.values, predictions)


            #deci = DecisionTreeClassifier(random_state=0,ccp_alpha=max(scores, key=scores.get)).fit(X_train, y_train)

            #models[deci] = max(scores.values())
            
            deci = DecisionTreeClassifier(random_state=20,ccp_alpha=best_ccp_alphas[colu]).fit(X_train, y_train)
            predictions = deci.predict(X_test)
            models[deci] = accuracy_score(y_test.values, predictions)

            simpl = np.full(y_test.shape, int(y_train.median()))
            models["simple"]=accuracy_score(y_test.values, simpl)
            best_model = max(models, key=models.get)


        else:
            print(colu,"Error: colu is neither in discrete nor in continous list")
            continue 

        print(colu,models)



        #final impute


        if best_model == "simple": #Check if simple median imputation is the best model
            print("Simple Imputation is better than the other models.")
            
        nan_pred = best_model.predict(simputed[simputed[colu].isnull()].drop(columns = colu))
        df_without_outliers.loc[df_without_outliers[colu].isnull(),colu] = nan_pred #impute the predictions
        
        
        summary[colu] = models
    #df_wo_outlier = pd.DataFrame(scaler.inverse_transform(df_wo_outlier),columns = df_wo_outlier.columns,index=df_wo_outlier.index) #bring all features back to the original scale
    rescaled_df = pd.DataFrame(scaler.inverse_transform(df_without_outliers[continous]),columns=continous,index=df_without_outliers[continous].index)
    df_without_outliers = pd.concat([rescaled_df, df_without_outliers[discrete]], axis=1)
    
    suma = pd.DataFrame(summary).reset_index()
    suma["index"]=suma["index"].astype("str")
    suma["index"]=suma["index"].str[:16]
    suma = suma.groupby("index").sum()
    suma.replace(0, np.nan, inplace=True)
    
    x = np.arange(len(nonmetric_cols))
    y1 = suma.loc["DecisionTreeClas",nonmetric_cols].dropna()
    y2 = suma.loc["LogisticRegressi",nonmetric_cols].dropna()
    y3 = suma.loc["simple",nonmetric_cols].dropna()

    width = 0.2

    # plot data in grouped manner of bar type
    plt.bar(x-0.2, y1, width, color='cyan')
    plt.bar(x, y2, width, color='orange')
    plt.bar(x+0.2, y3, width, color='green')
    plt.xticks(x, ['EducDeg', 'Children', 'GeoLivArea'])
    plt.ylabel("Score")
    plt.legend(["Decision Tree", "Logistic Regression", "Median Imputation"])
    plt.show()
    
    metric = ["BirthYear","MonthSal","PremMotor","PremHealth","PremLife","PremWork"]

    x = np.arange(suma.shape[1]-len(nonmetric_cols))
    y1 = suma.loc["KNeighborsRegres",metric].dropna()
    y2 = suma.loc["LinearRegression",metric].dropna()
    y3 = suma.loc["DecisionTreeRegr",metric].dropna()

    width = 0.2

    # plot data in grouped manner of bar type
    plt.bar(x-0.2, y1, width, color='cyan')
    plt.bar(x, y2, width, color='orange')
    plt.bar(x+0.2, y3, width, color='violet')

    plt.xticks(x, metric)
    plt.ylabel("Squared Error")
    plt.legend(["KNNRegressor", "Linear", "DecisionTree"])
    plt.show()
    
    
    pd.set_option('display.max_colwidth', None)
    best_met = pd.DataFrame(summary)[metric].idxmin()
    best_non_met = pd.DataFrame(summary)[nonmetric_cols].idxmax()

    best_models_for_each_feature=pd.concat([best_met, best_non_met], axis=0)
    imputed_df = df_without_outliers.copy()
    
    return imputed_df,best_models_for_each_feature

df_final_imputed,best_m = supervised_impute(misval_df)

### Prinicpal Component Analysis (PCA)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #cause most of our features follow approx. a normal distribution the most suitable scaler is the standardscaler
standard_scaled_df = pd.DataFrame(scaler.fit_transform(df_final_imputed[metric_cols]),columns=metric_cols,index=df_final_imputed[metric_cols].index)

from sklearn.decomposition import PCA
pca = PCA()
pca.fit(standard_scaled_df)
expl_varian = pca.explained_variance_ratio_ #Check how much variance is captured by each Principle component

cum_expl_varian = expl_varian.cumsum()

plt.figure(figsize=(6, 4))

plt.bar(range(1,10), expl_varian, alpha=0.5, align='center',
        label='individual explained variance')
plt.xticks(range(1,10))
plt.step(range(1,10), cum_expl_varian, where='mid',
         label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.show()

In [ ]:
# Output PCA table
a_pca_df = pd.DataFrame(
    {"Eigenvalue": pca.explained_variance_,
     "Difference": np.insert(np.diff(pca.explained_variance_), 0, 0),
     "Explained_Variance": pca.explained_variance_ratio_,
     "Cumulative": np.cumsum(pca.explained_variance_ratio_)},
    index=range(1, pca.n_components_ + 1)
)
a_pca_df

In [ ]:
optNr_comp = a_pca_df[(a_pca_df.Explained_Variance < 0.1) & (a_pca_df.Cumulative > 0.7)].index[0]-1

pca = PCA(n_components=3)
pca_feat = pca.fit_transform(df_final_imputed[metric_cols])
pca_feat_names = [f"PC{i+1}" for i in range(pca.n_components_)]
pca_df = pd.DataFrame(pca_feat, index=df_final_imputed.index, columns=pca_feat_names)  # remember index=df_pca.index
pca_df

In [ ]:
def vizualize_outlier_detect(df_before, df_after):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    
    from sklearn.decomposition import PCA
    pca = PCA(n_components=3)
    df= df_before[metric_cols].copy()
    for col in metric_cols:
        df[col].fillna(df[col].median(), inplace = True)
    #standard_scaled_df = pd.DataFrame(scaler.fit_transform(df[metric_cols]),columns=metric_cols,index=df[metric_cols].index)
    df_pca=pd.DataFrame(pca.fit_transform(df),columns=["PC1","PC2","PC3"],index=df[metric_cols].index)

    df_pca.loc[df_after.index,"Outlier?"]="No"
    df_pca.loc[[el for el in df_before.index if el not in df_after.index],"Outlier?"]="Yes"

    import plotly.express as px
    fig = px.scatter_3d(df_pca, x='PC1', y='PC2', z='PC3',
                  color='Outlier?',title="Visualization of Outliers using 3 Principle Components")
    
    fig.show()
    return df_pca
find_PC = vizualize_outlier_detect(df_final_imputed,df_final_imputed)

In [ ]:
# manually remove last outliers 
list_ID = find_PC[find_PC.PC2 > 5].index.to_list()
vizualize_outlier_detect(df_final_imputed[~df_final_imputed.index.isin(list_ID)],df_final_imputed[~df_final_imputed.index.isin(list_ID)])

In [ ]:
df_final_imputed

In [ ]:
# Reassigning df to contain pca variables
df_after_pca = pd.concat([df_final_imputed, pca_df], axis=1)
df_after_pca.head()

In [ ]:
def _color_red_or_green(val):
    if val < -0.45:
        color = 'background-color: red'
    elif val > 0.45:
        color = 'background-color: green'
    else:
        color = ''
    return color

# Interpreting each Principal Component
loadings = df_after_pca[metric_cols + pca_feat_names].corr().loc[metric_cols, pca_feat_names]
loadings.style.applymap(_color_red_or_green)

In [ ]:
def corr_heatmap(df,metric_columns,further_cols):
    df.corr()
    plt.figure(figsize=(15,15))
    corr = df[metric_columns + further_cols].corr()
    ax = sns.heatmap(
        corr, 
        vmin=-1, vmax=1, center=0,
        cmap=sns.diverging_palette(20, 220, n=200),
        square=True, annot=True
    )
    ax.set_xticklabels(
        ax.get_xticklabels(),
        rotation=45,
        horizontalalignment='right'
    )
    ax.set_yticklabels(
        ax.get_yticklabels(),
        rotation=45,
    );
    
corr_heatmap(df_after_pca,metric_cols,pca_feat_names)

#### Feature Engineering

In [ ]:
# as we mention in 3.4.2 we engineered 
Prem = ['PremMotor', 'PremHousehold',
       'PremHealth', 'PremLife', 'PremWork']
df_final_imputed.loc[:,"Prem/Income"]=df_final_imputed[Prem].sum(axis=1) / df_final_imputed.MonthSal
df_final_imputed.loc[:,"Profitable"]= np.where(df_final_imputed.CustMonVal>0,1,0)
df_final_imputed.loc[:,"Amount_paid_by_Insur_last_2_years"]= df_final_imputed.ClaimsRate * 2 * df_final_imputed[Prem].sum(axis=1)
#file.loc[:,"Age_first_purchase"]= file.FirstPolYear - file.BirthYear

### Feature Selection


In [ ]:
def corr_heatmap(df,metric_columns,further_cols):
    df.corr()
    plt.figure(figsize=(15,15))
    corr = df[metric_columns + further_cols].corr()
    ax = sns.heatmap(
        corr, 
        vmin=-1, vmax=1, center=0,
        cmap=sns.diverging_palette(20, 220, n=200),
        square=True, annot=True
    )
    ax.set_xticklabels(
        ax.get_xticklabels(),
        rotation=45,
        horizontalalignment='right'
    )
    ax.set_yticklabels(
        ax.get_yticklabels(),
        rotation=45,
    );







    
corr_heatmap(df_final_imputed,metric_cols,["Prem/Income","Profitable","Amount_paid_by_Insur_last_2_years"])

In [ ]:
# we will later use these functions to scale our data for clustering and evaluation
def Scale_Data_MINMAX(df):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    # transform data
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

    return df_scaled

In [ ]:
def Scale_Data_SS(df):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    # transform data
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

    return df_scaled

In [ ]:
# as explained in report 3.4.3 we removed features based on their correlation to each other and based on their own variance
scaled_imputed = Scale_Data_MINMAX(df_final_imputed)
secound_stage = scaled_imputed.describe().T[["std"]]
secound_stage["var"] = secound_stage**2
secound_stage.sort_values("var",ascending = False)

In [ ]:
# result of final Feature Selection
af_post_feat_sel = df_final_imputed.drop(["MonthSal","PremMotor","ClaimsRate","Prem/Income","Amount_paid_by_Insur_last_2_years"],1)


#metric_cols.append("MonthSal")
#metric_cols.append("ClaimsRate")
#metric_cols.append("PremMotor")
#metric_cols.append("Prem/Income")


metric_cols.remove("MonthSal")
metric_cols.remove("ClaimsRate")
metric_cols.remove("PremMotor")

nonmetric_cols.append("Profitable")

metric_cols_new = metric_cols

In [ ]:
# New Correlation Heatmap
corr_heatmap(df_final_imputed,metric_cols,["Prem/Income","Profitable"])

In [ ]:
# We found this code in the internet its based on the Paper "Spectral feature selection for supervised and unsupervised learning"
# But we didnt agreed with all results of it, therefor we didnt really used it and didnt mentioned it in our report 
import numpy.matlib
import numpy as np
from scipy.sparse import *
from sklearn.metrics.pairwise import rbf_kernel
from numpy import linalg as LA


def spec(X, **kwargs):
    """
    This function implements the SPEC feature selection
    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    kwargs: {dictionary}
        style: {int}
            style == -1, the first feature ranking function, use all eigenvalues
            style == 0, the second feature ranking function, use all except the 1st eigenvalue
            style >= 2, the third feature ranking function, use the first k except 1st eigenvalue
        W: {sparse matrix}, shape (n_samples, n_samples}
            input affinity matrix
    Output
    ------
    w_fea: {numpy array}, shape (n_features,)
        SPEC feature score for each feature
    Reference
    ---------
    Zhao, Zheng and Liu, Huan. "Spectral Feature Selection for Supervised and Unsupervised Learning." ICML 2007.
    """

    if 'style' not in kwargs:
        kwargs['style'] = 0
    if 'W' not in kwargs:
        kwargs['W'] = rbf_kernel(X, gamma=1)

    style = kwargs['style']
    W = kwargs['W']
    if type(W) is numpy.ndarray:
        W = csc_matrix(W)

    n_samples, n_features = X.shape

    # build the degree matrix
    X_sum = np.array(W.sum(axis=1))
    D = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        D[i, i] = X_sum[i]

    # build the laplacian matrix
    L = D - W
    d1 = np.power(np.array(W.sum(axis=1)), -0.5)
    d1[np.isinf(d1)] = 0
    d2 = np.power(np.array(W.sum(axis=1)), 0.5)
    v = np.dot(np.diag(d2[:, 0]), np.ones(n_samples))
    v = v/LA.norm(v)

    # build the normalized laplacian matrix
    L_hat = (np.matlib.repmat(d1, 1, n_samples)) * np.array(L) * np.matlib.repmat(np.transpose(d1), n_samples, 1)

    # calculate and construct spectral information
    s, U = np.linalg.eigh(L_hat)
    s = np.flipud(s)
    U = np.fliplr(U)

    # begin to select features
    w_fea = np.ones(n_features)*1000

    for i in range(n_features):
        f = X[:, i]
        F_hat = np.dot(np.diag(d2[:, 0]), f)
        l = LA.norm(F_hat)
        if l < 100*np.spacing(1):
            w_fea[i] = 1000
            continue
        else:
            F_hat = F_hat/l
        a = np.array(np.dot(np.transpose(F_hat), U))
        a = np.multiply(a, a)
        a = np.transpose(a)

        # use f'Lf formulation
        if style == -1:
            w_fea[i] = np.sum(a * s)
        # using all eigenvalues except the 1st
        elif style == 0:
            a1 = a[0:n_samples-1]
            w_fea[i] = np.sum(a1 * s[0:n_samples-1])/(1-np.power(np.dot(np.transpose(F_hat), v), 2))
        # use first k except the 1st
        else:
            a1 = a[n_samples-style:n_samples-1]
            w_fea[i] = np.sum(a1 * (2-s[n_samples-style: n_samples-1]))

    if style != -1 and style != 0:
        w_fea[w_fea == 1000] = -1000

    return w_fea


def feature_ranking(score, **kwargs):
    if 'style' not in kwargs:
        kwargs['style'] = 0
    style = kwargs['style']

    # if style = -1 or 0, ranking features in descending order, the higher the score, the more important the feature is
    if style == -1 or style == 0:
        idx = np.argsort(score, 0)
        return idx[::-1]
    # if style != -1 and 0, ranking features in ascending order, the lower the score, the more important the feature is
    elif style != -1 and style != 0:
        idx = np.argsort(score, 0)
        return idx

In [ ]:
score = spec(df_final_imputed.values)
print("The higher the score, the more important the feature is")
score

In [ ]:
import pandas as pd
import numpy as np

# Clustering

#### This Dataset is the final preprocessed Dataset:

In [ ]:
# Dataset including Feature selection
#af_post_feat_sel = pd.read_csv("data_ready_for_clustering.csv")
af_post_feat_sel

In [ ]:
#remaining metric_cols
metric_cols = ['BirthYear',
 'CustMonVal',
 'PremHousehold',
 'PremHealth',
 'PremLife',
 'PremWork']

In [ ]:
#remaining nonmetric_cols

nonmetric_cols = ['EducDeg', 'GeoLivArea', 'Children', 'Profitable']

## Stage One

#### Visualization Techniques

In [ ]:
# What I mentioned in the 4 captor of our report, its what we developed for the first stage evaluation of the models
# the code is pretty generic, we mostly scaled, fitted and transformed with TSNE, PCA or UMAP and then plotted it with plotly
# Visualization with PCA
def vizualize_cluster(df_with_cluster,metric_columns, nonmetic_columns,cluster_column):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    
    from sklearn.decomposition import PCA
    pca = PCA(n_components=3)
    df= df_with_cluster[metric_columns + nonmetic_columns].copy()
    
    standard_scaled_df = pd.DataFrame(scaler.fit_transform(df[metric_columns]),columns=metric_columns,index=df[metric_columns].index)
    df_pca=pd.DataFrame(pca.fit_transform(standard_scaled_df),columns=["PC1","PC2","PC3"],index=df[metric_columns].index)
    df_pca["cluster"] = df_with_cluster[cluster_column]

    import plotly.express as px
    fig = px.scatter_3d(df_pca, x='PC1', y='PC2', z='PC3',
                  color=cluster_column,title="Visualization of Outliers using 3 Principle Components")
    
    fig.show()


#Visualization with TSNE (unscaled)
def TSNE_Visualization(df,metric_features, cluster_col):
    from sklearn.manifold import TSNE
    import plotly.express as px
    from sklearn.preprocessing import StandardScaler
    # This is step can be quite time consuming
    TSNE1 = TSNE(random_state=38,n_components=2,perplexity = 18)
    scaler = MinMaxScaler()
    MinMax_scaled_df = pd.DataFrame(scaler.fit_transform(df[metric_cols]),columns=metric_cols,index=df[metric_cols].index)
    TSNE1 = TSNE1.fit_transform(MinMax_scaled_df[metric_features])
    some_df = df[[str(cluster_col)]].astype(int)

    fig = px.scatter(
    TSNE1, x=0, y=1,
    color = some_df[str(cluster_col)].astype("str"),width=900,
    height=900,labels={'color': 'cluster'},title="Dimensionreduction 2D with TSNE and MinMaxScaler")
    
    fig.show()

#Visualization with TSNE (unscaled)
def TSNE_Visualization_scaled(df,metric_features, cluster_col):
    from sklearn.manifold import TSNE
    import plotly.express as px
    from sklearn.preprocessing import StandardScaler
    # This is step can be quite time consuming
    TSNE1 = TSNE(random_state=38,n_components=3,perplexity = 18)
    scaler = MinMaxScaler()
    min_max_scaled_df = pd.DataFrame(scaler.fit_transform(df[metric_cols]),columns=metric_cols,index=df[metric_cols].index)
    TSNE1 = TSNE1.fit_transform(min_max_scaled_df[metric_features])
    some_df = df[[str(cluster_col)]].astype(int)

    fig = px.scatter_3d(
        TSNE1, x=0, y=1, z=2,
        color=some_df[str(cluster_col)],width=900,
        height=900,title="Dimensionreduction 3D with TSNE and MinMaxScaler",labels={'color': 'cluster'},)
    fig.update_traces(marker=dict(size=5,
                              line=dict(width=2,
                                        color='Black')),
                  selector=dict(mode='markers'))

    fig.show()

# UMAD Visualization (unscaled)    
def UMAD_Visualization(df,metric_features, cluster_col):
    import umap.umap_ as umap
    import plotly.express as px
    # This is step can be quite time consuming
    UMAP1 = umap.UMAP(n_components=2, init='random', random_state=38)
    scaler = MinMaxScaler()
    min_max_scaled_df = pd.DataFrame(scaler.fit_transform(df[metric_cols]),columns=metric_cols,index=df[metric_cols].index)
    UMAP1 = UMAP1.fit_transform(min_max_scaled_df[metric_features])
    some_df = df[[str(cluster_col)]].astype(int)

    fig = px.scatter(
    UMAP1, x=0, y=1,
    color = some_df[str(cluster_col)].astype("str"),width=900,
    height=900,labels={'color': 'cluster'},title="Dimensionreduction 2D with UMAD and MinMaxScaler")
    
    fig.show()    

    
# UMAD Visualization (Standardscaled)
def UMAD_Visualization_StandardScaled(df,metric_features, cluster_col):
    import umap.umap_ as umap
    import plotly.express as px
    from sklearn.preprocessing import StandardScaler
    # This is step can be quite time consuming
    UMAP1 = umap.UMAP(n_components=3, init='random', random_state=38)
    scaler = StandardScaler()
    standard_scaled_df = pd.DataFrame(scaler.fit_transform(df[metric_cols]),columns=metric_cols,index=df[metric_cols].index)
    UMAP1 = UMAP1.fit_transform(standard_scaled_df[metric_features])
    some_df = df[[str(cluster_col)]].astype(int)

    fig = px.scatter_3d(
        UMAP1, x=0, y=1, z=2,
        color=some_df[str(cluster_col)].astype("str"),width=900,
        height=900,labels={'color': 'cluster'},title="Dimensionreduction with UMAD and StandardScaler"
    )
    fig.show()

# UMAD Visualization (MinMaxscaled)
def UMAD_Visualization_MinMaxScaled(df,metric_features, cluster_col):
    import umap.umap_ as umap
    import plotly.express as px
    from sklearn.preprocessing import MinMaxScaler
    # This is step can be quite time consuming
    UMAP1 = umap.UMAP(n_components=3, init='random', random_state=38)
    scaler = MinMaxScaler()
    min_max_scaled_df = pd.DataFrame(scaler.fit_transform(df[metric_cols]),columns=metric_cols,index=df[metric_cols].index)
    UMAP1 = UMAP1.fit_transform(min_max_scaled_df[metric_features])
    some_df = df[[str(cluster_col)]].astype(int)

    fig = px.scatter_3d(
        UMAP1, x=0, y=1, z=2,
        color=some_df[str(cluster_col)].astype("str"),width=900,
        height=900,labels={'color': 'cluster'},title="Dimensionreduction with UMAD and MinMaxScaler")
    fig.update_traces(marker=dict(size=5,
                              line=dict(width=2,
                                        color='Black')),
                  selector=dict(mode='markers'))
    fig.show()

In [ ]:
# related to 4.1 First Stage our Report
visual_df = af_post_feat_sel.copy()
visual_df["cluster"] = [2 for x in range(len(visual_df))]
UMAD_Visualization(visual_df,metric_cols, "cluster")
TSNE_Visualization(visual_df,metric_cols, "cluster")

TSNE_Visualization_scaled(visual_df,metric_cols, "cluster")
UMAD_Visualization_MinMaxScaled(visual_df,metric_cols, "cluster")

### Silhouette and BIC

In [ ]:
# We need this later to see the best number of clusters for different algorithms
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from sklearn.mixture import GaussianMixture as GMM
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import rcParams

def SelBest(arr:list, X:int)->list:
    '''
    returns the set of X configurations with shorter distance
    '''
    dx=np.argsort(arr)[:X]
    return arr[dx]



#### Agglomerative Clustering

In [ ]:
#Source: https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html

import numpy as np

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering


def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

X = Scale_Data_MINMAX(af_post_feat_sel[metric_cols])

linkage = ["ward", "complete", "average", "single"]
for method in linkage:

    # setting distance_threshold=0 ensures we compute the full tree.
    model = AgglomerativeClustering(distance_threshold=0, n_clusters=None,linkage = method)

    model = model.fit(X)
    plt.figure(figsize=(8, 6))
    plt.title(f"Hierarchical Clustering Dendrogram:{method} linkage")
    # plot the top six levels of the dendrogram
    plot_dendrogram(model, truncate_mode="level", p=3)
    #plt.hlines(6.3, 0, 2000, colors="r", linestyles="dashed")
    plt.xlabel("Number of points in node (or index of point if no parenthesis).")
    plt.show()
    print("____________________________\n")

In [ ]:
#Source: Practical Class
def get_r2_hc(df, link_method, max_nclus, min_nclus=1, dist="euclidean"):
    """This function computes the R2 for a set of cluster solutions given by the application of a hierarchical method.
    The R2 is a measure of the homogenity of a cluster solution. It is based on SSt = SSw + SSb and R2 = SSb/SSt. 
    
    Parameters:
    df (DataFrame): Dataset to apply clustering
    link_method (str): either "ward", "complete", "average", "single"
    max_nclus (int): maximum number of clusters to compare the methods
    min_nclus (int): minimum number of clusters to compare the methods. Defaults to 1.
    dist (str): distance to use to compute the clustering solution. Must be a valid distance. Defaults to "euclidean".
    
    Returns:
    ndarray: R2 values for the range of cluster solutions
    """
    def get_ss(df):
        ss = np.sum(df.var() * (df.count() - 1))
        return ss  # return sum of sum of squares of each df variable
    
    sst = get_ss(df)  # get total sum of squares
    
    r2 = []  # where we will store the R2 metrics for each cluster solution
    
    for i in tqdm(range(min_nclus, max_nclus+1)):  # iterate over desired ncluster range
        cluster = AgglomerativeClustering(n_clusters=i, affinity=dist, linkage=link_method)
        
        
        hclabels = cluster.fit_predict(df) #get cluster labels
        
        
        df_concat = pd.concat((df, pd.Series(hclabels, name='labels')), axis=1)  # concat df with labels
        
        
        ssw_labels = df_concat.groupby(by='labels').apply(get_ss)  # compute ssw for each cluster labels
        
        
        ssb = sst - np.sum(ssw_labels)  # remember: SST = SSW + SSB
        
        
        r2.append(ssb / sst)  # save the R2 of the given cluster solution
        
    return np.array(r2)

In [ ]:
# Prepare input
hc_methods = ["ward", "complete", "average", "single"]
# Call function defined above to obtain the R2 statistic for each hc_method
max_nclus = 11
r2_hc_methods = np.vstack(
    [
        get_r2_hc(df=X[metric_cols], link_method=link, max_nclus=max_nclus) 
        for link in hc_methods
    ]
).T
r2_hc_methods = pd.DataFrame(r2_hc_methods, index=range(1, max_nclus + 1), columns=hc_methods)

sns.set()
# Plot data
fig = plt.figure(figsize=(11,5))
sns.lineplot(data=r2_hc_methods, linewidth=2.5, markers=["o"]*4)

# Finalize the plot
fig.suptitle("R2 plot for various hierarchical methods", fontsize=21)
plt.gca().invert_xaxis()  # invert x axis
plt.legend(title="HC methods", title_fontsize=11)
plt.xticks(range(1, max_nclus + 1))
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R2 metric", fontsize=13)

plt.show()

In [ ]:
#final hierachical clustering 
df_AGGLO = af_post_feat_sel.copy()
agglomerative_c = AgglomerativeClustering(n_clusters=3,linkage = "ward")
labels_agglomerative = agglomerative_c.fit_predict(MinMaxScaler().fit_transform(df_AGGLO[metric_cols]))
df_AGGLO.loc[:,"cluster"] = labels_agglomerative
#UMAD_Visualization_StandardScaled(df_AGGLO,metric_cols,"cluster")
#Visualization of our results
TSNE_Visualization_scaled(df_AGGLO,metric_cols,"cluster")
UMAD_Visualization_MinMaxScaled(df_AGGLO,metric_cols,"cluster")
df_AGGLO

### GaussianMixture

In [ ]:

import matplotlib.cm as cm
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.mixture import GaussianMixture

# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py
#using this to get the best nr of clusters
# Storing average silhouette metric
df = Scale_Data_MINMAX(af_post_feat_sel).copy()
metric_features = metric_cols
range_clusters = range(1, 11)

avg_silhouette = []
for nclus in range_clusters:
    # Skip nclus == 1
    if nclus == 1:
        continue
    
    # Create a figure
    fig = plt.figure(figsize=(13, 7))

    # Initialize the KMeans object with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    GMMclust = GaussianMixture(n_components=nclus,tol=0.5)
    cluster_labels = GMMclust.fit_predict(df[metric_features])

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the densyity and separation of the formed clusters
    silhouette_avg = silhouette_score(df[metric_features], cluster_labels)
    avg_silhouette.append(silhouette_avg)
    print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df[metric_features], cluster_labels)

    y_lower = 10
    for i in range(nclus):
        # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        # Get y_upper to demarcate silhouette y range size
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        # Filling the silhouette
        color = cm.nipy_spectral(float(i) / nclus)
        plt.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    plt.title("The silhouette plot for the various clusters.")
    plt.xlabel("The silhouette coefficient values")
    plt.ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    plt.axvline(x=silhouette_avg, color="red", linestyle="--")
    
    # The silhouette coefficient can range from -1, 1
    xmin, xmax = np.round(sample_silhouette_values.min() -0.1, 2), np.round(sample_silhouette_values.max() + 0.1, 2)
    plt.xlim([xmin, xmax])
    
    # The (nclus+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    plt.ylim([0, len(df[metric_features]) + (nclus + 1) * 10])

    plt.yticks([])  # Clear the yaxis labels / ticks
    plt.xticks(np.arange(xmin, xmax, 0.1))

In [ ]:
# no clue where I found this code, but its not mine ;-)
# anyways here we just visualize the optimal number of clusters using GaussianMixture as Model
df_copy = af_post_feat_sel.copy()
n_clusters=np.arange(2, 25)
sils=[]
sils_err=[]
iterations=8
for n in n_clusters:
    tmp_sil=[]
    for _ in tqdm(range(iterations)):
        gmm=GaussianMixture(n_components=n).fit(Scale_Data_MINMAX(df_copy)[metric_cols]) 
        labels=gmm.predict(Scale_Data_MINMAX(df_copy)[metric_cols])
        sil=metrics.silhouette_score(Scale_Data_MINMAX(df_copy)[metric_cols], labels, metric='euclidean')
        tmp_sil.append(sil)
    val=np.mean(SelBest(np.array(tmp_sil), int(iterations/5)))
    err=np.std(tmp_sil)
    sils.append(val)
    sils_err.append(err)

plt.errorbar(n_clusters, sils, yerr=sils_err)
plt.title("Silhouette Scores", fontsize=20)
plt.xticks(n_clusters)
plt.xlabel("N. of clusters")
plt.ylabel("Score")

In [ ]:
# parts of the search for the best hyperparameter, we did the same with TOL and Reg_covar
df_copy = af_post_feat_sel.copy()
some_dict = dict()
for _ in tqdm(range(180,200)):
    gmm=GaussianMixture(n_components=3,random_state = _,tol = 0.5,reg_covar = 1e-5,max_iter = 400,init_params = "random").fit(Scale_Data_MINMAX(df_copy)[metric_cols]) 
    labels=gmm.predict(Scale_Data_MINMAX(df_copy)[metric_cols])
    sil=metrics.silhouette_score(Scale_Data_MINMAX(df_copy)[metric_cols], labels, metric='euclidean')
    some_dict[_] = sil

In [ ]:
#Example
gmm=GaussianMixture(n_components=3,random_state = 179,tol = 1).fit(Scale_Data_MINMAX(df_copy)[metric_cols]) 
labels=gmm.predict(Scale_Data_MINMAX(df_copy)[metric_cols])
sil=metrics.silhouette_score(Scale_Data_MINMAX(df_copy)[metric_cols], labels, metric='euclidean')
sil

In [ ]:
#Example
gmm=GaussianMixture(n_components=3,random_state = 186,tol = 0.1,reg_covar = 1e-5,max_iter = 400).fit(Scale_Data_MINMAX(df_copy)[metric_cols]) 
labels=gmm.predict(Scale_Data_MINMAX(df_copy)[metric_cols])
sil=metrics.silhouette_score(Scale_Data_MINMAX(df_copy)[metric_cols], labels, metric='euclidean')
sil

In [ ]:
#result of Loop above
max(some_dict,key = some_dict.get)

In [ ]:
# training gaussian mixture model 
# final solution and visualization
from sklearn.mixture import GaussianMixture
df_GMM = af_post_feat_sel.copy()

GMM = GaussianMixture(n_components=3,random_state = 179,tol=0.5,reg_covar = 1e-5)

#predictions from gmm

labels_GMM = GMM.fit_predict(Scale_Data_MINMAX(df_GMM)[metric_cols])
df_GMM.loc[:,"cluster"] = labels_GMM
TSNE_Visualization_scaled(df_GMM,metric_cols,"cluster")
UMAD_Visualization_MinMaxScaled(df_GMM,metric_cols,"cluster")
vizualize_cluster(df_GMM,metric_cols, nonmetric_cols,"cluster")
df_GMM

In [ ]:
# secound stage evaluation of different models
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

metrices= ["Silhouette(max)","Calinski-Harabasz(max)","Davies-Bouldin(min)"]
clustmethod= ["KMeans","DBScan","Gaussian Mixture"]
evalu = pd.DataFrame(np.zeros((len(metrices),len(clustmethod))),index=metrices,columns=clustmethod)

def evaluation(df,cluster_label,clmethod_name):
    evalu.loc[metrices,clmethod_name]=[silhouette_score(df,cluster_label)
                                       ,calinski_harabasz_score(df, cluster_label)
                                       ,davies_bouldin_score(df,cluster_label)]
    
evaluation(Scale_Data_MINMAX(af_post_feat_sel)[metric_cols],labels_GMM,"Gaussian Mixture_3_tol_scaled_1e-5_final")
evalu

## Spectral Clustering

In [ ]:

import matplotlib.cm as cm
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import SpectralClustering

# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py
# same we did for GMM we do it for spectral as well - results are more or less the same
# Storing average silhouette metric
df = Scale_Data_MINMAX(af_post_feat_sel).copy()
metric_features = metric_cols
range_clusters = range(1, 11)

avg_silhouette = []
for nclus in range_clusters:
    # Skip nclus == 1
    if nclus == 1:
        continue
    
    # Create a figure
    fig = plt.figure(figsize=(13, 7))

    # Initialize the KMeans object with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    SPECclust = SpectralClustering(n_components=nclus,assign_labels='discretize')
    cluster_labels = SPECclust.fit_predict(df[metric_features])

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters
    silhouette_avg = silhouette_score(df[metric_features], cluster_labels)
    avg_silhouette.append(silhouette_avg)
    print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df[metric_features], cluster_labels)

    y_lower = 10
    for i in range(nclus):
        # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        # Get y_upper to demarcate silhouette y range size
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        # Filling the silhouette
        color = cm.nipy_spectral(float(i) / nclus)
        plt.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    plt.title("The silhouette plot for the various clusters.")
    plt.xlabel("The silhouette coefficient values")
    plt.ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    plt.axvline(x=silhouette_avg, color="red", linestyle="--")
    
    # The silhouette coefficient can range from -1, 1
    xmin, xmax = np.round(sample_silhouette_values.min() -0.1, 2), np.round(sample_silhouette_values.max() + 0.1, 2)
    plt.xlim([xmin, xmax])
    
    # The (nclus+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    plt.ylim([0, len(df[metric_features]) + (nclus + 1) * 10])

    plt.yticks([])  # Clear the yaxis labels / ticks
    plt.xticks(np.arange(xmin, xmax, 0.1))

In [ ]:
df_SPEC = af_post_feat_sel.copy()
n_clusters=np.arange(2, 7)
sils=[]
sils_err=[]
iterations=8
for n in n_clusters:
    tmp_sil=[]
    for _ in tqdm(range(iterations)):
        SPEC = SpectralClustering(n_components=n,assign_labels='discretize')
        labels=SPEC.fit_predict(Scale_Data_MINMAX(df_SPEC)[metric_cols])
        sil=metrics.silhouette_score(Scale_Data_MINMAX(df_SPEC)[metric_cols], labels, metric='euclidean')
        tmp_sil.append(sil)
    val=np.mean(SelBest(np.array(tmp_sil), int(iterations/5)))
    err=np.std(tmp_sil)
    sils.append(val)
    sils_err.append(err)
plt.errorbar(n_clusters, sils, yerr=sils_err)
plt.title("Silhouette Scores", fontsize=20)
plt.xticks(n_clusters)
plt.xlabel("N. of clusters")
plt.ylabel("Score")

In [ ]:
#! pip install pyamg
# training spectral model 
from sklearn.cluster import SpectralClustering
df_SPEC = af_post_feat_sel.copy()
SPEC = SpectralClustering(n_components=3,assign_labels = "discretize",eigen_solver = "arpack")

#predictions from SPEC

labels_SPEC = SPEC.fit_predict(Scale_Data_MINMAX(df_SPEC)[metric_cols])
df_SPEC.loc[:,"cluster"] = labels_SPEC
UMAD_Visualization_MinMaxScaled(df_SPEC,metric_cols,"cluster")
TSNE_Visualization_scaled(df_SPEC,metric_cols,"cluster")
vizualize_cluster(df_SPEC,metric_cols, nonmetric_cols,"cluster")
df_SPEC

In [ ]:
evalu

In [ ]:
# training spectral model 
from sklearn.cluster import SpectralClustering
df_SPEC2 = af_post_feat_sel.copy()
SPEC = SpectralClustering(n_components=3,assign_labels='discretize')

#predictions from SPEC

labels_SPEC = SPEC.fit_predict(Scale_Data_MINMAX(df_SPEC2)[metric_cols])
df_SPEC2.loc[:,"cluster"] = labels_SPEC
UMAD_Visualization_StandardScaled(df_SPEC2,metric_cols,"cluster")
UMAD_Visualization_MinMaxScaled(df_SPEC2,metric_cols,"cluster")
vizualize_cluster(df_SPEC2,metric_cols, nonmetric_cols,"cluster")
df_SPEC2

### OPTICS

In [ ]:
# training Optics
from sklearn.cluster import OPTICS
df_OPT = af_post_feat_sel.copy()
OPT = OPTICS(min_samples=4,eps = 0.0001,metric = "euclidean")

#predictions from OPT

labels_OPT = OPT.fit_predict(Scale_Data_MINMAX(df_OPT[metric_cols]))
df_OPT.loc[:,"cluster"] = labels_OPT
UMAD_Visualization_StandardScaled(df_OPT,metric_cols,"cluster")
UMAD_Visualization_MinMaxScaled(df_OPT,metric_cols,"cluster")
df_OPT

### KMeans

In [ ]:

import matplotlib.cm as cm
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import KMeans

# Adapted from:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

# Storing average silhouette metric
df = af_post_feat_sel.copy()
metric_features = metric_cols
range_clusters = range(1, 11)

avg_silhouette = []
for nclus in range_clusters:
    # Skip nclus == 1
    if nclus == 1:
        continue
    
    # Create a figure
    fig = plt.figure(figsize=(13, 7))

    # Initialize the KMeans object with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    kmclust = KMeans(n_clusters=nclus, init='k-means++', n_init=15, random_state=1)
    cluster_labels = kmclust.fit_predict(df[metric_features])

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters
    silhouette_avg = silhouette_score(df[metric_features], cluster_labels)
    avg_silhouette.append(silhouette_avg)
    print(f"For n_clusters = {nclus}, the average silhouette_score is : {silhouette_avg}")

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df[metric_features], cluster_labels)

    y_lower = 10
    for i in range(nclus):
        # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        # Get y_upper to demarcate silhouette y range size
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        # Filling the silhouette
        color = cm.nipy_spectral(float(i) / nclus)
        plt.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    plt.title("The silhouette plot for the various clusters.")
    plt.xlabel("The silhouette coefficient values")
    plt.ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    plt.axvline(x=silhouette_avg, color="red", linestyle="--")
    
    # The silhouette coefficient can range from -1, 1
    xmin, xmax = np.round(sample_silhouette_values.min() -0.1, 2), np.round(sample_silhouette_values.max() + 0.1, 2)
    plt.xlim([xmin, xmax])
    
    # The (nclus+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    plt.ylim([0, len(df[metric_features]) + (nclus + 1) * 10])

    plt.yticks([])  # Clear the yaxis labels / ticks
    plt.xticks(np.arange(xmin, xmax, 0.1))

In [ ]:
from sklearn.cluster import KMeans
df_KMEANS = af_post_feat_sel.copy()
KMEANS1 = KMeans(n_clusters=4,init = "k-means++" ,random_state=38,n_init = 100)


# training and prediction KMEANS

labels_KMEANS = KMEANS1.fit_predict(Scale_Data_MINMAX(df_KMEANS))
df_KMEANS.loc[:,"cluster"] = labels_KMEANS

UMAD_Visualization_StandardScaled(df_KMEANS,metric_cols,"cluster")
UMAD_Visualization_MinMaxScaled(df_KMEANS,metric_cols,"cluster")
vizualize_cluster(df_KMEANS,metric_cols, nonmetric_cols,"cluster")
df_KMEANS

### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

#For hyperparameter min_sample we choose the rule of thumb (number of features -1) = 5.
#Tune hypterparameter epsilon for dbscan: 
df_DB = af_post_feat_sel.copy()

for epsil in tqdm(np.arange(0.001, 0.3, 0.001)):

    db = DBSCAN(
      eps = epsil,
      metric="euclidean",
      min_samples = 5,
      n_jobs = -1)
    clusters = db.fit_predict(Scale_Data_MINMAX(df_DB)[metric_cols])



    unique, counts = np.unique(clusters, return_counts=True)
    dbscan = dict(zip(unique, counts))

    for key,value in dbscan.copy().items():

        #We want to ignore tiny clusters with less than 40 data points and outliers with the label -1.
        if (value < 40) or (key == -1):
            del dbscan[key]
    #Only consider epsilons that result in at least 3 clusters of significant size.
    if len(dbscan) > 2:
        print(epsil,dbscan)
        
#We choose an epsilon of 0.056 as we get at least 2 clusters of signifcant size

db = DBSCAN(
      eps = 0.056,
      metric="euclidean",
      min_samples = 5,
      n_jobs = -1)
labels_dbscan = db.fit_predict(Scale_Data_MINMAX(df_DB)[metric_cols])   
df_DB.loc[:,"cluster"] = labels_dbscan
UMAD_Visualization_MinMaxScaled(df_DB,metric_cols,"cluster")

#DB Scan is working really bad for our dataset. Reason for that is that the point density is not decreasing between the clusters. Clusters in our dataset are by no means center of high density.

### Birch

In [ ]:
from sklearn.cluster import Birch

df_BRCH = af_post_feat_sel.copy()
BRCH = Birch(n_clusters=3,branching_factor =150)




# training and prediction BIRCH

labels_BRCH = BRCH.fit_predict(Scale_Data_MINMAX(df_KMEANS))
df_BRCH.loc[:,"cluster"] = labels_BRCH

UMAD_Visualization_StandardScaled(df_BRCH,metric_cols,"cluster")
UMAD_Visualization_MinMaxScaled(df_BRCH,metric_cols,"cluster")
vizualize_cluster(df_BRCH,metric_cols, nonmetric_cols,"cluster")
df_BRCH

### Evaluation Methods

In [ ]:
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

metrices= ["Silhouette(max)","Calinski-Harabasz(max)","Davies-Bouldin(min)"]
clustmethod= ["KMeans","DBScan","Gaussian Mixture"]
evalu = pd.DataFrame(np.zeros((len(metrices),len(clustmethod))),index=metrices,columns=clustmethod)
# Compute three evalluation metrices for the cluster labels 
def evaluation(df,cluster_label,clmethod_name):
    evalu.loc[metrices,clmethod_name]=[silhouette_score(df,cluster_label)
                                       ,calinski_harabasz_score(df, cluster_label)
                                       ,davies_bouldin_score(df,cluster_label)]

# DF is MinMaxScaled
evaluation(Scale_Data_MINMAX(af_post_feat_sel)[metric_cols],labels_agglomerative,"Hierachical")
evaluation(Scale_Data_MINMAX(af_post_feat_sel)[metric_cols],labels_GMM,"Gaussian Mixture")
evaluation(Scale_Data_MINMAX(af_post_feat_sel)[metric_cols],labels_SPEC,"Spectral Clustering_3")
evaluation(Scale_Data_MINMAX(af_post_feat_sel)[metric_cols],labels_OPT,"OPTICS")
evaluation(Scale_Data_MINMAX(af_post_feat_sel)[metric_cols],labels_KMEANS,"KMeans")
evaluation(Scale_Data_MINMAX(af_post_feat_sel)[metric_cols],labels_dbscan,"DBScan")
evaluation(Scale_Data_MINMAX(af_post_feat_sel)[metric_cols],labels_BRCH,"BIRCH")


evalu

In [ ]:
#IF the DF is standardscaled
evaluation(Scale_Data_SS(af_post_feat_sel)[metric_cols],labels_agglomerative,"Hierachical")
evaluation(Scale_Data_SS(af_post_feat_sel)[metric_cols],labels_GMM,"Gaussian Mixture")
evaluation(Scale_Data_SS(af_post_feat_sel)[metric_cols],labels_SPEC,"Spectral Clustering")
evaluation(Scale_Data_SS(af_post_feat_sel)[metric_cols],labels_OPT,"OPTICS")
evaluation(Scale_Data_SS(af_post_feat_sel)[metric_cols],labels_KMEANS,"KMeans")
evaluation(Scale_Data_SS(af_post_feat_sel)[metric_cols],labels_dbscan,"DBScan")
evaluation(Scale_Data_SS(af_post_feat_sel)[metric_cols],labels_BRCH,"BIRCH")



evalu

In [ ]:
n_clusters=np.arange(2, 10)
bics=[]
bics_err=[]
iterations=20
for n in n_clusters:
    tmp_bic=[]
    for _ in tqdm(range(iterations)):
        gmm=GaussianMixture(n_components=n).fit(af_post_feat_sel) 
        
        tmp_bic.append(gmm.bic(af_post_feat_sel))
    val=np.mean(SelBest(np.array(tmp_bic), int(iterations/5)))
    err=np.std(tmp_bic)
    bics.append(val)
    bics_err.append(err)

In [ ]:
plt.errorbar(n_clusters,bics, yerr=bics_err, label='BIC')
plt.title("BIC Scores", fontsize=20)
plt.xticks(n_clusters)
plt.xlabel("N. of clusters")
plt.ylabel("Score")
plt.legend()

In [ ]:
# Optimal NR of Cluster with ElbowMethod and K-Means

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib.style as style

range_n_clusters = [1, 2, 3, 4, 5, 6]
avg_distance=[]
for n_clusters in range(1,20):
    clusterer = KMeans(n_clusters=int(n_clusters), random_state=42).fit(af_post_feat_sel)
    avg_distance.append(clusterer.inertia_)

style.use("fivethirtyeight")
plt.plot(range(1,20), avg_distance)
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Distance")
plt.show()

## Conclusion

In [ ]:
#df_SPEC = df_SPEC.set_index("CustID")

#Compute the feature means of each cluster to comparte the characteristics of each cluster. We want to include all features for that.
df_SPEC_all = pd.merge(df_SPEC[["cluster"]],file, left_index=True, right_index=True,how='left')
df_SPEC_all.drop(["Missing_Values"],axis = 1,inplace = True)
df_SPEC_all.groupby("cluster").mean().sort_values("BirthYear")

In [ ]:
df_GMM.groupby("cluster").mean()

In [ ]:
#df_GMM = df_GMM.set_index("CustID")
df_GMM_all = pd.merge(df_GMM[["cluster"]],file, left_index=True, right_index=True,how='left')
df_GMM_all.drop(["Missing_Values"],axis = 1,inplace = True)
df_GMM_all.groupby("cluster").mean().sort_values("BirthYear")

In [ ]:
#df_AGGLO = df_AGGLO.set_index("CustID")
df_aggl_all = pd.merge(df_AGGLO[["cluster"]],file, left_index=True, right_index=True,how='left')
df_aggl_all.drop(["Missing_Values"],axis = 1,inplace = True)
df_aggl_all.groupby("cluster").mean().sort_values("BirthYear")

## Classification of Outliers using KNN

In [ ]:
outlier

In [ ]:
#We now build a classifier using KNN to assign a cluster to each outlier detected. We find the best number of K, which is 32 in out case.

X_pred = Scale_Data_MINMAX(outlier[metric_cols])
y = df_SPEC_all["cluster"].astype("int")
X = Scale_Data_MINMAX(df_final_imputed[metric_cols])

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

scores=list()
for k in tqdm(range(2,40)):

    neigh = KNeighborsClassifier(n_neighbors=k)
    scores.append(cross_val_score(neigh, X.values, y, cv=10).mean())
best_k = scores.index(max(scores)) + 1
print(f"The best k from 1 to 40 is: {best_k}.")

neigh = KNeighborsClassifier(n_neighbors=best_k).fit(X,y)
#Using cross validation to avoid overfitting
print(f"The cross validation accuracy score is {cross_val_score(neigh, X.values, y, cv=10).mean()}.")
y_pred= neigh.predict(X_pred)

outlier = file.loc[outlier.index,:].drop(columns="Missing_Values")
outlier.loc[:,"cluster"]= y_pred

final_clustered_df = pd.concat([outlier,df_SPEC_all]) # Add the outliers along with their predictions back to the other data points.

In [ ]:
!pip install -U git+https://github.com/sevamoo/SOMPY.git

import sompy
from sompy.visualization.mapview import View2D
from sompy.visualization.bmuhits import BmuHitsView
from sompy.visualization.hitmap import HitMapView
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:


df_scaled_som = Scale_Data_MINMAX(af_post_feat_sel[metric_cols]).copy()



In [ ]:
np.random.seed(42)

sm = sompy.SOMFactory().build(
    df_scaled_som.values, 
    mapsize=[10, 10],
    initialization='random', 
    neighborhood='gaussian',
    training='batch',
    lattice='hexa',
    component_names=metric
)
sm.train(n_job=4, verbose='info', train_rough_len=100, train_finetune_len=100)

In [ ]:
sns.set()
view2D = View2D(12, 12, "", text_size=10)
view2D.show(sm, col_sz=3, what='codebook')
plt.subplots_adjust(top=0.90)
plt.suptitle("Component Planes", fontsize=20)
plt.show()

In [ ]:
# Here you have U-matrix
u = sompy.umatrix.UMatrixView(9, 9, 'umatrix-Avg Distance from each Neuron to its Neighbor', show_axis=True, text_size=8, show_text=True)

UMAT = u.show(
    sm, 
    distance=2, 
    row_normalized=False,
    show_data=True, 
    contour=True, # Visualize isomorphic curves
    blob=False
)

In [ ]:
vhts  = BmuHitsView(12,12,"Hits Map - Number of Points that each Neuron represent")
vhts.show(sm, anotate=True, onlyzeros=False, labelsize=12, cmap="Blues")
plt.show()